scikit-learn은 파이썬에서 머신러닝을 구현할 때 사용하는 라이브러리입니다.

scikit-learn에서 제공하는 머신러닝 알고리즘을 가져와 사용합니다.
- xgboost와 lightgbm은 scikit-learn에서 제공하지 않습니다.
1. Logistic Regression
2. PLSRegression
3. LinearDiscriminantAnalysis
4. QuadraticDiscriminantAnalysis
5. DecisionTreeClassifier
6. RandomForestClassifier
7. GradientBoostingClassifier
8. xgboost
9. lightgbm
10. MLPClassifier


parameter_grid(common 항목 참조)는 scikit-learn에서 제공하는 머신러닝 알고리즘의 하이퍼파라미터를 설정하는 파이썬 파일입니다.

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, 
    QuadraticDiscriminantAnalysis
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

import numpy as np

from module.common import parameter_grid

load_model은 모델명과 파라미터를 받아서 모델을 생성하는 함수입니다. 모델명은 문자열로 받고, 파라미터는 딕셔너리로 받습니다. 파라미터는 모델마다 다르기 때문에 딕셔너리로 받아서 모델에 넣어줍니다.

In [ ]:
def load_model(model: str, seed: int, param: dict):
    if model == 'logistic':
        clf = LogisticRegression(random_state = seed, **param)
        
    elif model == 'dt':
        clf = DecisionTreeClassifier(random_state = seed, **param)
    
    elif model == 'rf':
        clf = RandomForestClassifier(random_state = seed, **param)
    
    elif model == 'gbt':
        clf = GradientBoostingClassifier(random_state = seed, **param)
    
    elif model == 'xgb':
        clf = XGBClassifier(random_state = seed, **param)
    
    elif model == 'lgb':
        clf = LGBMClassifier(random_state = seed, **param)
    
    elif model == 'lda':
        clf = LinearDiscriminantAnalysis(**param)
    
    elif model == 'qda':
        clf = QuadraticDiscriminantAnalysis(**param)
    
    elif model == 'plsda':
        clf = PLSRegression(**param)
        
    elif model == 'mlp':
        clf = MLPClassifier(random_state = seed, **param)
    
    return clf


load_hyperparameter는 모델마다 하이퍼파라미터를 설정해주는 함수입니다. 모델의 이름을 문자열로 받아 해당되는 모델에 적합한 하이퍼파라미터를 반환합니다.

lda의 경우 solver에 따라서 하이퍼파라미터가 달라지기 때문에 두 가지 경우를 모두 고려해줍니다.
1. lsqr과 eigen의 경우 shrinkage값을 포함합니다.
2. svd의 경우 tol값을 포함합니다.

dictionary로 구성된 하이퍼파라미터를 parameter_grid(common 항목 참조) 함수를 통해 하이퍼파라미터 조합을 생성합니다.

In [ ]:
def load_hyperparameter(model: str):
    if model == 'logistic':
        params_dict = {
            'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 
                  1, 2, 3, 4, 5, 7, 9, 11, 15, 20, 25, 30, 35, 40, 50, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga']
        }
        
    elif model == 'dt':
        params_dict = {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 1, 2, 3, 4, 5],
            'min_samples_split': [2, 3, 4],
            'min_samples_leaf': [1, 2, 3]
        }
    
    elif model == 'rf':
        params_dict = {
            'n_estimators': [3, 5, 10, 15, 20, 30, 50, 90, 95, 
                            100, 125, 130, 150],
            'criterion': ['gini'],
            'min_samples_split': [2, 4],
            'min_samples_leaf': [1, 3],
            'max_features': ['sqrt', 'log2']
        }
    
    elif model == 'gbt':
        params_dict = {
            'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1],
            'n_estimators': [5, 10, 50, 100, 130],
            'max_depth': [1, 2, 3, 4]
        }
    
    elif model == 'xgb':
        params_dict = {
        'min_child_weight': [1, 2, 3, 5],
            'max_depth': [3, 6, 9],
            'gamma': np.linspace(0, 3, 10),
            # 'objective': ['multi:softmax'],
            'booster': ['gbtree']
        }
    
    elif model == 'lgb':
        params_dict = {
            # 'objective': ['multiclass'],
            'num_leaves': [15, 21, 27, 31, 33],
            'max_depth': [-1, 2],
            'n_estimators': [5, 10, 50, 100, 130],
            'min_child_samples': [10, 20, 25, 30]
        }
    
    elif model == 'lda':
        params_dict1 = {
            'solver': ['lsqr', 'eigen'],
            'shrinkage': np.logspace(-3, 0, 30)
        }
        params_dict2 = {
            'solver': ['svd'],
            'tol': np.logspace(-5, -3, 20)
        }
    
    elif model == 'qda':
        params_dict = {
            'reg_param': np.append(np.array([0]), np.logspace(-5, 0, 10)),
            'tol': np.logspace(-5, -3, 10)
        }
    
    elif model == 'plsda':
        params_dict = {
            'n_components': [1, 2, 3],
            'max_iter': [300, 500, 1000],
            'tol': np.logspace(-7, -5, 10)
        }
        
    elif model == 'mlp':
        params_dict = {
            'hidden_layer_sizes': [(50), (100, 50, 10), (100, 70, 50, 30, 10)],
            'activation': ['relu', 'tanh'],
            'solver': ['adam', 'sgd'],
            'alpha': [0.0001, 0.001],
            'learning_rate_init': [0.001, 0.01, 0.1],
            'max_iter': [50, 100, 200]
        }
    
    #
    if model == 'lda':
        params = parameter_grid(params_dict1)
        params.extend(parameter_grid(params_dict2))
    else:
        params = parameter_grid(params_dict)
    
    return params